<a href="https://colab.research.google.com/github/zealpatel1990/KDM_spring_2021_class/blob/main/ICP%203/source_code/ICP_3_zeal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install stanza, Installing and importing Stanza are as simple as running the following commands. 
!pip install stanza

# Import stanza
import stanza

In [ ]:
# Download the Stanford CoreNLP package with Stanza's installation command
# This'll take several minutes, depending on the network speed
corenlp_dir = './corenlp'
stanza.install_corenlp(dir=corenlp_dir)

# Set the CORENLP_HOME environment variable to point to the installation location
import os
os.environ["CORENLP_HOME"] = corenlp_dir

2021-02-04 20:48:34 INFO: Installing CoreNLP package into ./corenlp...
2021-02-04 20:51:01 WARNING: For customized installation location, please set the `CORENLP_HOME` environment variable to the location of the installation. In Unix, this is done with `export CORENLP_HOME=./corenlp`.


In [ ]:
import stanza
# Import client module
from stanza.server import CoreNLPClient

client = CoreNLPClient(timeout=150000000, be_quiet=True, annotators=['openie'], 
endpoint='http://localhost:9003')
client.start()
import time
time.sleep(10)

2021-02-04 21:28:10 INFO: Writing properties to tmp file: corenlp_server-5443ac7c6cb1439d.props
2021-02-04 21:28:10 INFO: Starting server with command: java -Xmx5G -cp ./corenlp/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9003 -timeout 150000000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-5443ac7c6cb1439d.props -annotators openie -preload -outputFormat serialized


In [ ]:
import json
s='Twenty percent electric motors are pulled from an assembly line'
s1='Brack Obama was born in Hawaii'
text = "Albert Einstein was a German-born theoretical physicist. He developed the theory of relativity."
document = client.annotate(s1, output_format='json')
triples = []
for sentence in document['sentences']:
    for i in sentence:
      for triple in sentence['openie']:
        triples.append({
           'subject': triple['subject'],
           'relation': triple['relation'],
            'object': triple['object']
        })
print(triples)

[{'subject': 'Brack Obama', 'relation': 'was born in', 'object': 'Hawaii'}, {'subject': 'Brack Obama', 'relation': 'was', 'object': 'born'}, {'subject': 'Brack Obama', 'relation': 'was born in', 'object': 'Hawaii'}, {'subject': 'Brack Obama', 'relation': 'was', 'object': 'born'}, {'subject': 'Brack Obama', 'relation': 'was born in', 'object': 'Hawaii'}, {'subject': 'Brack Obama', 'relation': 'was', 'object': 'born'}, {'subject': 'Brack Obama', 'relation': 'was born in', 'object': 'Hawaii'}, {'subject': 'Brack Obama', 'relation': 'was', 'object': 'born'}, {'subject': 'Brack Obama', 'relation': 'was born in', 'object': 'Hawaii'}, {'subject': 'Brack Obama', 'relation': 'was', 'object': 'born'}, {'subject': 'Brack Obama', 'relation': 'was born in', 'object': 'Hawaii'}, {'subject': 'Brack Obama', 'relation': 'was', 'object': 'born'}]


In [ ]:
result = [document["sentences"][0]["openie"] for item in document]
for i in result:
    for rel in i:
        relationSent=rel['relation'],rel['subject'],rel['object']
        print('The triplet of the given sentence is')
        print(relationSent)

The triplet of the given sentence is
('was born in', 'Brack Obama', 'Hawaii')
The triplet of the given sentence is
('was', 'Brack Obama', 'born')


In [ ]:
import nltk
nltk.download('all')
from nltk.corpus import wordnet

In [ ]:
# lets use word paint as an exqmple
syns = wordnet.synsets("good")

# An example of a synset:
print(syns[0].name())
print('\n')
# Just the word:
print(syns[0].lemmas()[0].name())
print('\n')

# Definition of that first synset:
print(syns[0].definition())
print('\n')
# Examples of the word in use in sentences:
print(syns[0].examples())
print('\n')

# synonyms and antonyms using wordnet using word
synonyms = []
antonyms = []

for syn in wordnet.synsets("good"):
    for l in syn.lemmas():
        synonyms.append(l.name())
        if l.antonyms():
            antonyms.append(l.antonyms()[0].name())
print('The synonyms of good are: ')
print(set(synonyms))
print('\n')
print('The antonyms of good are: ')
print(set(antonyms))
print('\n')



# comparison/ similarity score between 2 words
w1 = wordnet.synset('ship.n.01')
w2 = wordnet.synset('boat.n.01') # n denotes noun
print("The similarity score between ship and boat is =",w1.wup_similarity(w2))

good.n.01


good


benefit


['for your own good', "what's the good of worrying?"]


The synonyms of good are: 
{'skilful', 'honorable', 'dear', 'respectable', 'practiced', 'dependable', 'sound', 'soundly', 'near', 'undecomposed', 'honest', 'unspoiled', 'proficient', 'estimable', 'commodity', 'secure', 'effective', 'salutary', 'beneficial', 'just', 'serious', 'goodness', 'ripe', 'expert', 'upright', 'in_effect', 'good', 'skillful', 'right', 'thoroughly', 'full', 'safe', 'well', 'unspoilt', 'trade_good', 'in_force', 'adept'}


The antonyms of good are: 
{'ill', 'bad', 'evilness', 'evil', 'badness'}


The similarity score between ship and boat is = 0.9090909090909091


# ICP Tasks:

### 1: IE: Triplet Extraction

In [113]:
import spacy
from spacy.lang.en import English
import networkx as nx
import matplotlib.pyplot as plt

# spliting input text in list of words
def getSentences(text):
    nlp = English()
    nlp.add_pipe(nlp.create_pipe('sentencizer'))
    document = nlp(text)
    return [sent.string.strip() for sent in document.sents]

# Function to print word and token
def printToken(token):
    print(token.text, "->", token.dep_)

def appendChunk(original, chunk):
    return original + ' ' + chunk

def isRelationCandidate(token):
    deps = ["ROOT", "adj", "attr", "agent", "amod"]
    return any(subs in token.dep_ for subs in deps)

def isConstructionCandidate(token):
    deps = ["compound", "prep", "conj", "mod"]
    return any(subs in token.dep_ for subs in deps)

# This function will pass subject, object, relation, subject construction and object contruction and print triplets of the sentence
def processSubjectObjectPairs(tokens):
    subject = ''
    object = ''
    relation = ''
    subjectConstruction = ''
    objectConstruction = ''
    for token in tokens:
        printToken(token)
        if "punct" in token.dep_:
            continue
        if isRelationCandidate(token):
            relation = appendChunk(relation, token.lemma_)
        if isConstructionCandidate(token):
            if subjectConstruction:
                subjectConstruction = appendChunk(subjectConstruction, token.text)
            if objectConstruction:
                objectConstruction = appendChunk(objectConstruction, token.text)
        if "subj" in token.dep_:
            subject = appendChunk(subject, token.text)
            subject = appendChunk(subjectConstruction, subject)
            subjectConstruction = ''
        if "obj" in token.dep_:
            object = appendChunk(object, token.text)
            object = appendChunk(objectConstruction, object)
            objectConstruction = ''

  # printing triplets of given sentence
    print ("\nThe triplet of the given sentence is: \nSubject: ", subject.strip(),
            ",\nRelation: ", relation.strip(),
            ",\nObject: ", object.strip())
    return (subject.strip(), relation.strip(), object.strip())

# This returns object pairs from the sentence by using NLP_model function
def processSentence(sentence):
    tokens = nlp_model(sentence)
    return processSubjectObjectPairs(tokens)

In [114]:

#sample text
text = "Gandhiji was born in Porbandar."

sentences = getSentences(text)
nlp_model = spacy.load('en_core_web_sm')

triples = []
print ("Input --> ", text)
# printing the relation and construction candidate
for sentence in sentences:
    triples.append(processSentence(sentence))

#Born is the past participle of the verb bear

Input -->  Gandhiji was born in Porbandar.
Gandhiji -> nsubjpass
was -> auxpass
born -> ROOT
in -> prep
Porbandar -> pobj
. -> punct

The triplet of the given sentence is: 
Subject:  Gandhiji ,
Relation:  bear ,
Object:  Porbandar


### 2: WordNet Task:

In [17]:
import nltk
nltk.download('wordnet')

from nltk.corpus import wordnet as wn

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


#### 1.Hyponym 

In [67]:
vehicle_synset = wn.synset('vehicle.n.01')
# getting sorted list of HYPONYMS from vehicle_synset
vehicle_synset.hyponyms()
# somethiing like child words

[Synset('bumper_car.n.01'),
 Synset('craft.n.02'),
 Synset('military_vehicle.n.01'),
 Synset('rocket.n.01'),
 Synset('skibob.n.01'),
 Synset('sled.n.01'),
 Synset('steamroller.n.02'),
 Synset('wheeled_vehicle.n.01')]

#### 2.Hypernym 

In [31]:
vehicle_synset = wn.synset('projectile.n.2')
# getting sorted list of HYPONYMS from vehicle_synset
sorted([lemma.name() for synset in vehicle_synset.hypernyms() for lemma in synset.lemmas()])

['vehicle']

#### 3.Meronym

In [57]:
animal_synset = wn.synset('animal.n.01')
# getting sorted list of HYPONYMS from vehicle_synset
print(animal_synset.substance_meronyms())
print(animal_synset.part_meronyms())


[Synset('animal_tissue.n.01')]
[Synset('face.n.07'), Synset('head.n.01')]


#### 4.Holonym

In [70]:
human_synset = wn.synset('human.n.01')
# hyponyms for human
human_synset.hyponyms()

[Synset('homo_erectus.n.01'),
 Synset('homo_habilis.n.01'),
 Synset('homo_sapiens.n.01'),
 Synset('homo_soloensis.n.01'),
 Synset('neandertal_man.n.01'),
 Synset('rhodesian_man.n.01'),
 Synset('world.n.08')]

#### 5.Entailment 

In [93]:
stand_synset = wn.synset('stand.v.01')
# entailment for verb stand
stand_synset.entailments()

[Synset('arise.v.03')]